In [15]:
!pip install mlflow boto3 awscli

In [17]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-35-92-223-149.us-west-2.compute.amazonaws.com:8000/")

In [18]:
mlflow.set_experiment("Exp 4 - Handling Imbalanced Data")

2025/10/01 04:23:39 INFO mlflow.tracking.fluent: Experiment with name 'Exp 4 - Handling Imbalanced Data' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-damini/862200131645839431', creation_time=1759292619438, experiment_id='862200131645839431', last_update_time=1759292619438, lifecycle_stage='active', name='Exp 4 - Handling Imbalanced Data', tags={}>

In [19]:
from google.colab import files

uploaded = files.upload()

Saving reddit_preprocessing.xls to reddit_preprocessing (1).xls


In [20]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv('reddit_preprocessing.xls')
df.head()

,clean_comment,category
0,family mormon have never tried explain them th...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [22]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [23]:
def run_imbalanced_experiment(imbalance_method):
  ngram_range=(1,3)
  max_features=1000
  X_train,X_test,y_train,y_test= train_test_split(df['clean_comment'],df['category'],random_state=42,test_size=0.2,stratify=df['category'])
  vectorizer= TfidfVectorizer(ngram_range=ngram_range,max_features=max_features)
  X_train_vec=vectorizer.fit_transform(X_train)
  X_test_vec=vectorizer.transform(X_test)

  if imbalance_method == 'class_weights':
    class_weight = 'balanced'
  else:
     class_weight = None  # Do not apply class_weight if using resampling
     if imbalance_method == 'oversampling':
      smote=SMOTE(random_state=42)
      X_train_vec,y_train=smote.fit_resample(X_train_vec,y_train)
     elif imbalance_method == 'undersampling':
        rus=RandomUnderSampler(random_state=42)
        X_train_vec,y_train=rus.fit_resample(X_train_vec,y_train)
     elif imbalance_method == 'adasyn':
        adasyn=ADASYN(random_state=42)
        X_train_vec,y_train=adasyn.fit_resample(X_train_vec,y_train)
     elif imbalance_method == 'smote_enn':
        smote_enn = SMOTEENN(random_state=42)
        X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

# Step 5: Define and train a Random Forest model
  with mlflow.start_run() as run:
    mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
    mlflow.set_tag("experiment_type", "imbalance_handling")
    mlflow.set_tag("model_type", "RandomForestClassifier")
    mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}")

    mlflow.log_param("vectorizer_type", "TF-IDF")
    mlflow.log_param("ngram_range", ngram_range)
    mlflow.log_param("vectorizer_max_features", max_features)
    n_estimators = 200
    max_depth = 15
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("imbalance_method", imbalance_method)

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)

    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)
    classification_rep = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
    confusion_matrix_filename = f"confusion_matrix_{imbalance_method}.png"
    plt.savefig(confusion_matrix_filename)
    mlflow.log_artifact(confusion_matrix_filename)
    plt.close()

        # Log the model
    mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")


imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_imbalanced_experiment(method)

2025/10/01 04:25:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/01 04:25:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Trigrams at: http://ec2-35-92-223-149.us-west-2.compute.amazonaws.com:8000/#/experiments/862200131645839431/runs/061922fa992c46d6b4efdc2b6ea4edc3
🧪 View experiment at: http://ec2-35-92-223-149.us-west-2.compute.amazonaws.com:8000/#/experiments/862200131645839431


2025/10/01 04:26:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/01 04:26:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Trigrams at: http://ec2-35-92-223-149.us-west-2.compute.amazonaws.com:8000/#/experiments/862200131645839431/runs/427e385151c94575b6ccff85cab51821
🧪 View experiment at: http://ec2-35-92-223-149.us-west-2.compute.amazonaws.com:8000/#/experiments/862200131645839431


2025/10/01 04:27:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/01 04:27:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Trigrams at: http://ec2-35-92-223-149.us-west-2.compute.amazonaws.com:8000/#/experiments/862200131645839431/runs/465d7d25914148ac9a9a4c77ec07a653
🧪 View experiment at: http://ec2-35-92-223-149.us-west-2.compute.amazonaws.com:8000/#/experiments/862200131645839431


2025/10/01 04:28:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/01 04:28:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Trigrams at: http://ec2-35-92-223-149.us-west-2.compute.amazonaws.com:8000/#/experiments/862200131645839431/runs/6de649c634da4a90916001a608459a92
🧪 View experiment at: http://ec2-35-92-223-149.us-west-2.compute.amazonaws.com:8000/#/experiments/862200131645839431


2025/10/01 04:29:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/01 04:29:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Trigrams at: http://ec2-35-92-223-149.us-west-2.compute.amazonaws.com:8000/#/experiments/862200131645839431/runs/6a4ada8c546d4a2d83217038a3aaa819
🧪 View experiment at: http://ec2-35-92-223-149.us-west-2.compute.amazonaws.com:8000/#/experiments/862200131645839431
